In [3]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import statsmodels.api as sm

In [2]:
path_ref = 'C:\\Users\\tnguyen10\\OneDrive - Deloitte (O365D)\\Documents\\GitHub\\Thesis\\data\\refinitiv'

In [4]:
df_sample = pd.read_excel(os.path.join(path_ref,'sample.xlsx'))
df_sentiment = pd.read_csv('sentiment_scores.csv') #drop h&m and dhl
df_similarity = pd.read_csv('similarity_scores.csv')

In [7]:
df_sample

,company,resource use,emissions,environmental innovation,strategy
0,abb,62.39,99.39,36.68,62.18
1,adidas,82.77,90.63,80.68,76.89
2,airbus,99.43,97.09,95.92,83.49
3,apple,98.51,97.50,43.51,87.06
4,bayer,70.38,87.91,82.94,95.75
...,...,...,...,...,...
59,veolia,74.49,79.00,84.15,69.16
60,volkswagen,97.25,96.49,92.69,96.07
61,volvo,96.45,88.41,95.18,87.55
62,walmart,77.73,88.08,76.81,87.01


In [9]:
df_sentiment

,company,0,1,2,3,4,5,6,total
0,henkel,-0.007491,-0.007629,0.012023,0.013756,0.004118,0.002563,-0.030222,-0.012884
1,general-mills,NaN,0.013966,-0.005795,0.019676,0.009944,0.004485,0.007125,0.049402
2,enel,0.000759,-0.018271,0.006222,0.001978,0.040981,0.019511,0.000442,0.051621
3,airbus,0.023659,0.003928,0.009152,-0.006690,0.046551,0.013631,0.009065,0.099296
4,eversource,0.005571,0.065910,0.009005,0.013049,0.077035,0.022740,0.000113,0.193422
...,...,...,...,...,...,...,...,...,...
58,dupont,0.006339,0.011108,0.008123,0.000057,0.081427,0.015871,0.031506,0.154431
59,mcdonald,-0.004740,0.000351,0.027431,0.019437,0.074865,0.019405,0.044869,0.181618
60,colgate,-0.002326,0.009628,0.007202,0.014026,0.021664,0.011243,0.060053,0.121489
61,nestle,-0.000185,0.000620,0.029275,0.013297,0.020408,0.046627,0.029277,0.139318


In [10]:
df_similarity

,company,0,1,2,3,4,5,6,average,rev_similarity
0,henkel,0.307193,0.441082,0.316931,0.471574,0.252742,0.515490,0.552531,0.408221,0.485468
1,general-mills,NaN,0.401772,0.317858,0.511924,0.268937,0.500265,0.260843,0.376933,0.603152
2,enel,0.307359,0.529861,0.247507,0.491178,0.324345,0.490653,0.330006,0.388701,0.558887
3,airbus,0.577002,0.485288,0.334677,0.580605,0.338507,0.485650,0.380358,0.454584,0.311079
4,eversource,0.257189,0.513299,0.231859,0.432165,0.351334,0.472154,0.213468,0.353067,0.692922
...,...,...,...,...,...,...,...,...,...,...
58,dupont,0.360307,0.412970,0.300349,0.470131,0.375814,0.574513,0.432614,0.418100,0.448309
59,mcdonald,0.358209,0.507266,0.480616,0.542530,0.401509,0.529817,0.588919,0.486981,0.189221
60,colgate,0.309629,0.231054,0.277209,0.442338,0.230000,0.505484,0.535156,0.361553,0.661002
61,nestle,0.255926,0.441334,0.359633,0.505296,0.327689,0.506255,0.540308,0.419492,0.443073


In [8]:
df_sentiment.rename(columns = {'Unnamed: 0': 'company'}, inplace = True)
df_similarity.rename(columns = {'Unnamed: 0': 'company'}, inplace = True)

In [11]:
df_sentiment['total'] = MinMaxScaler().fit_transform(np.array(df_sentiment['total']).reshape(-1,1))

### Preparing the Refinitiv data

In [12]:
df_sample['average_practice'] = df_sample[['resource use', 'emissions', 'environmental innovation']].replace(0, np.nan).mean(axis = 1)

In [14]:
df_sample['green_practice'] = StandardScaler().fit_transform(np.array(df_sample['average_practice']).reshape(-1,1))
df_sample['green_communication'] = StandardScaler().fit_transform(np.array(df_sample['strategy']).reshape(-1,1))
df_sample['discrepancy_index'] = df_sample['green_communication']-df_sample['green_practice']

In [17]:
df_linreg = pd.merge(df_sentiment, df_sample, how = 'left')

In [20]:
df_linreg['rev_similarity'] = df_similarity['rev_similarity']

In [22]:
df_linreg['discrepancy_index'] = MinMaxScaler().fit_transform(np.array(df_linreg['discrepancy_index']).reshape(-1,1))

In [34]:
df_linreg.set_index('company', inplace = True)

In [35]:
df_linreg.drop(['h&m', 'apple', 'dhl'], inplace = True)

In [36]:
df_linreg

,0,1,2,3,4,5,6,total,resource use,emissions,environmental innovation,strategy,average_practice,green_practice,green_communication,discrepancy_index,rev_similarity
company,,,,,,,,,,,,,,,,,
henkel,-0.007491,-0.007629,0.012023,0.013756,0.004118,0.002563,-0.030222,0.326188,99.53,82.43,27.62,95.75,69.860000,-0.944383,0.678493,0.763328,0.485468
general-mills,NaN,0.013966,-0.005795,0.019676,0.009944,0.004485,0.007125,0.423352,98.55,78.95,21.53,97.03,66.343333,-1.235565,0.768897,0.815389,0.603152
enel,0.000759,-0.018271,0.006222,0.001978,0.040981,0.019511,0.000442,0.426814,98.82,94.18,95.59,90.07,96.196667,1.236307,0.277326,0.411078,0.558887
airbus,0.023659,0.003928,0.009152,-0.006690,0.046551,0.013631,0.009065,0.501186,99.43,97.09,95.92,83.49,97.480000,1.342568,-0.187407,0.333176,0.311079
eversource,0.005571,0.065910,0.009005,0.013049,0.077035,0.022740,0.000113,0.648022,99.83,96.38,69.16,87.01,88.456667,0.595431,0.061204,0.469029,0.692922
hp,0.003819,0.001629,-0.013664,-0.005359,0.006226,-0.001639,-0.002801,0.327894,96.57,71.78,43.59,97.03,70.646667,-0.879247,0.768897,0.766775,0.258974
chipotle,NaN,-0.057568,0.040197,-0.008214,0.026651,0.035018,-0.069959,0.293442,53.35,42.66,67.35,39.71,54.453333,-2.220063,-3.279505,0.397372,0.448754
danone,NaN,-0.001235,0.020724,0.012976,0.058509,0.048212,0.065822,0.666096,79.29,95.83,85.40,98.12,86.840000,0.461570,0.845882,0.594347,0.278315
bayer,0.002133,0.006421,0.010472,-0.013869,0.007225,0.008552,-0.009305,0.364427,70.38,87.91,82.94,95.75,80.410000,-0.070838,0.678493,0.644148,0.593190


In [37]:
X = df_linreg['total']
y = df_linreg['discrepancy_index']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:      discrepancy_index   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.702
Date:                Thu, 04 May 2023   Prob (F-statistic):              0.197
Time:                        21:21:13   Log-Likelihood:                 17.490
No. Observations:                  60   AIC:                            -30.98
Df Residuals:                      58   BIC:                            -26.79
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6407      0.079      8.066      0.0

In [38]:
X = df_linreg['rev_similarity']
y = df_linreg['discrepancy_index']
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:      discrepancy_index   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.599
Date:                Thu, 04 May 2023   Prob (F-statistic):              0.211
Time:                        21:21:18   Log-Likelihood:                 17.438
No. Observations:                  60   AIC:                            -30.88
Df Residuals:                      58   BIC:                            -26.69
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.6027      0.054     11.